In [ ]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import logging

# Configuration
SYMBOL = "NQH26.CME"
PERIOD = "200d"
INTERVAL = "60m"
NY_TZ = "America/New_York"
OUTPUT_DIR = Path("../data/processed")
OUTPUT_FILENAME = "nq626_4h_fixed.xlsx"

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_ohlc(symbol: str, period=PERIOD, interval=INTERVAL) -> pd.DataFrame:
    try:
        logging.info(f"Downloading data for {symbol}...")
        df = yf.download(symbol, period=period, interval=interval,
                        auto_adjust=True, progress=False, group_by="column")
        if df is None or df.empty:
            raise RuntimeError(f"Download failed: {symbol}")
        
        if df.index.tz is None:
            df = df.tz_localize("UTC")
        df = df.tz_convert(NY_TZ)
        
        # Compatibility for possible multi-level columns
        if isinstance(df.columns, pd.MultiIndex):
            if symbol in df.columns.get_level_values(0):
                df = df.xs(symbol, axis=1, level=0)
            else:
                df = df.xs(symbol, axis=1, level=1)
        
        logging.info(f"Successfully downloaded {len(df)} rows for {symbol}")
        return df[['Open','High','Low','Close']]
    except Exception as e:
        logging.error(f"Error downloading data: {e}")
        raise

def resample_4h_et18(ohlc: pd.DataFrame) -> pd.DataFrame:
    logging.info("Resampling data to 4H...")
    return (ohlc
            .resample('4h', origin='start_day', offset='18h')
            .agg({'Open':'first','High':'max','Low':'min','Close':'last'})
            .dropna())

def main():
    try:
        ohlc = download_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL)
        k4 = resample_4h_et18(ohlc)

        out = (k4.tail(200)
                  .rename(columns={'Open':'4H_Open','High':'4H_High',
                                   'Low':'4H_Low','Close':'4H_Close'}))
        out.index = out.index.tz_localize(None)
        out.index.name = 'Datetime'

        # Ensure output directory exists
        OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        output_path = OUTPUT_DIR / OUTPUT_FILENAME
        
        out.reset_index().to_excel(output_path, index=False)
        logging.info(f"[Version1] Using contract {SYMBOL}, exported to: {output_path}")
        
    except Exception as e:
        logging.critical(f"Process failed: {e}")

if __name__ == '__main__':
    main()